In [1]:
import numpy as np
import torch
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected = True)

## 3.1 Автоматическое дифференцирование в `torch`

3.1.1 Воспользовавшись классами `Neuron` и `SquaredLoss` из задачи 2.4.1 и автоматическим дифференцированием, которое предоставляет `torch`, решить задачу регрессии. Для оптимизации использовать стохастический градиетный спуск.

In [2]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)

In [3]:
class Neuron:
    def __init__(self, n_inputs):
        self.weights = torch.randn(n_inputs, 1, requires_grad=True)
        self.bias = torch.randn(1, 1, requires_grad=True)

    def forward(self, inputs):
        return inputs @ self.weights + self.bias

    def backward(self):
        self.dweights = self.weights.grad
        self.dbias = self.bias.grad


In [4]:
class SquaredLoss:
    def forward(self, y_pred, y_true):
        self.loss = (y_pred - y_true).pow(2).mean()
        return self.loss

    def backward(self):
        self.loss.backward()


In [5]:
neuron = Neuron(4)
loss = SquaredLoss()

for epoch in range(100):
    for x_example, y_example in zip(X, y):
        y_pred = neuron.forward(x_example)
        loss.forward(y_pred, y_example)
        loss.backward()
        neuron.backward()
        with torch.no_grad():
            neuron.weights -= 0.01 * neuron.dweights
            neuron.bias -= 0.01 * neuron.dbias
            neuron.weights.grad.zero_()
            neuron.bias.grad.zero_()
    
    if epoch % 30 == 0:
        print(f"Epoch {epoch}: {loss.loss}")

print(neuron.weights.T, neuron.bias)
print(coef)

Epoch 0: 1129.6588134765625
Epoch 30: 2.0954757928848267e-09
Epoch 60: 2.0954757928848267e-09
Epoch 90: 2.0954757928848267e-09
tensor([[60.3878, 12.3079, 95.2505, 77.7915]], grad_fn=<PermuteBackward0>) tensor([[0.5000]], requires_grad=True)
[60.38775633 12.30793488 95.25045104 77.79149303]


3.1.2 Воспользовавшись классами `Linear` и `MSELoss` из задачи 2.1.4 и 2.3.1, `ReLU` из 2.2.1 и автоматическим дифференцированием, которое предоставляет `torch`, решить задачу регрессии. Для оптимизации использовать пакетный градиентный спуск. Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные значения.

In [6]:
class Linear:
    def __init__(self, n_features, n_neurons): 
        self.weights = torch.randn(n_features, n_neurons, requires_grad=True)
        self.biases = torch.randn(n_neurons, requires_grad=True)
 
    def forward(self, inputs):
        return inputs @ self.weights + self.biases # <реализовать логику слоя>

    def backward(self):
        self.dweights = self.weights.grad
        self.dbiases = self.biases.grad


class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs.clamp(min=0)
        return self.output

    def backward(self):
        self.dinputs = self.inputs.clamp(min=0)

class MSELoss:
    def forward(self, y_pred, y_true):
        self.loss = (y_pred.flatten() - y_true.flatten()).pow(2).mean()
        return self.loss

    def backward(self):
        self.loss.backward()

In [7]:
X, _, coef = make_regression(n_features=1, n_informative=1, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
# y = torch.from_numpy(y).to(dtype=torch.float32)
y = torch.sin(X)

In [9]:
fc1 = Linear(1, 512)
relu1 = Activation_ReLU()
fc2 = Linear(512, 1)

loss = MSELoss()
lr = 0.001
epochs = 20001

ys = []
losses = []
for epoch in range(epochs):
    out = fc2.forward(relu1.forward(fc1.forward(X)))
    data_loss = loss.forward(out, y)
    losses.append(data_loss.detach())
    
    if epoch % 2000 == 0:
        print(f'epoch {epoch} mean loss {data_loss}')
        ys.append(out.flatten())

    loss.backward()
    fc2.backward()
    fc1.backward()

    with torch.no_grad():
        fc1.weights -= lr * fc1.dweights
        fc1.biases -= lr * fc1.dbiases
        fc2.weights -= lr * fc2.dweights
        fc2.biases -= lr * fc2.dbiases

        fc1.weights.grad.zero_()
        fc1.biases.grad.zero_()
        fc2.weights.grad.zero_()
        fc2.biases.grad.zero_()
        
        

epoch 0 mean loss 134.85572814941406
epoch 2000 mean loss 0.0016873711720108986
epoch 4000 mean loss 0.0009014481329359114
epoch 6000 mean loss 0.0005843310500495136
epoch 8000 mean loss 0.00042419126839376986
epoch 10000 mean loss 0.00032618624391034245
epoch 12000 mean loss 0.0002687912783585489
epoch 14000 mean loss 0.00023144528677221388
epoch 16000 mean loss 0.00020462066459003836
epoch 18000 mean loss 0.0001846393570303917
epoch 20000 mean loss 0.0001689775090198964


In [10]:
px.line(x=range(epochs), y=losses)

In [11]:
for out in ys:
    fig = px.scatter(x = X.flatten().numpy(), y = y.flatten().numpy())
    fig.add_scatter(x = X.flatten().numpy(), y = out.detach().numpy(), mode='markers')
    fig.show()

## 3.2 Алгоритмы оптимизации в `torch.optim`

3.2.1 Решить задачу 3.1.1, воспользовавшись оптимизатором `optim.SDG` для применения стохастического градиентого спуска

In [12]:
X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)

neuron = Neuron(4)
loss = SquaredLoss()
optimizer = torch.optim.SGD([neuron.weights, neuron.bias], lr=0.01)

for epoch in range(100):
    for x_example, y_example in zip(X, y):
        y_pred = neuron.forward(x_example)
        loss.forward(y_pred, y_example)
        loss.backward()
        neuron.backward()
        with torch.no_grad():
            optimizer.step()
            neuron.weights.grad.zero_()
            neuron.bias.grad.zero_()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: {loss.loss}")

print(neuron.weights.T.data.numpy(), neuron.bias.data.numpy())
print(coef)

Epoch 0: 220.08126831054688
Epoch 20: 2.0954757928848267e-09
Epoch 40: 2.0954757928848267e-09
Epoch 60: 2.0954757928848267e-09
Epoch 80: 2.0954757928848267e-09
[[75.47804   8.189229 80.25234  48.233875]] [[0.499994]]
[75.478058    8.18922843 80.25233423 48.23387895]


3.2.2 Решить задачу 3.1.2, воспользовавшись оптимизатором `optim.Adam` для применения пакетного градиентого спуска. Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные значения.

In [13]:
X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)
y = y / y.abs().max()  # Сходимость очень долгая если значения y большие 

neuron = Neuron(4)
loss = SquaredLoss()
optimizer = torch.optim.Adam([neuron.weights, neuron.bias])

for epoch in range(100):
    for x_example, y_example in zip(X, y):
        y_pred = neuron.forward(x_example)
        loss.forward(y_pred, y_example)
        loss.backward()
        neuron.backward()
        with torch.no_grad():
            optimizer.step()
            neuron.weights.grad.zero_()
            neuron.bias.grad.zero_()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: {loss.loss}")

print(neuron.weights.T.data.numpy(), neuron.bias.data.numpy())
print(coef)

Epoch 0: 0.8400236368179321
Epoch 20: 0.47574374079704285
Epoch 40: 0.18288862705230713
Epoch 60: 0.026727186515927315
Epoch 80: 0.00017225570627488196
[[0.03961803 0.09801274 0.28805405 0.1584821 ]] [[0.00177582]]
[11.21986738 27.75551658 81.58730436 44.88848694]


## 3.3 Построение сетей при помощи `torch.nn`

3.3.1 Решить задачу регрессии, соблюдая следующие условия:

1. Оформить нейронную сеть в виде класса - наследника `nn.Module`
2. При создании сети использовать готовые блоки из `torch.nn`: слои, функции активации, функции потерь и т.д.
3. Для оптимизации использовать любой алгоритм оптимизации из `torch.optim` 

In [14]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())

In [15]:
class Model331(torch.nn.Module):
    def __init__(self, input_size: int = 1, hidden_size: int = 512, output_size: int = 1):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.activation1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        return x

    
model311 = Model331()
optimizer = torch.optim.SGD(model311.parameters(), 0.01)
loss = torch.nn.MSELoss()

for epoch in range(10000):

    y_pred = model311(X)
    data_loss = loss(y_pred.flatten(), y.flatten())
    data_loss.backward()

    with torch.no_grad():
        optimizer.step()
    
    model311.zero_grad()
            
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: {data_loss.item()}")


Epoch 0: 0.7480062246322632
Epoch 500: 0.13736288249492645
Epoch 1000: 0.08386162668466568
Epoch 1500: 0.04741150513291359
Epoch 2000: 0.02678235061466694
Epoch 2500: 0.016003044322133064
Epoch 3000: 0.010473073460161686
Epoch 3500: 0.007581512443721294
Epoch 4000: 0.00597429508343339
Epoch 4500: 0.0049768597818911076
Epoch 5000: 0.004282377194613218
Epoch 5500: 0.003745080204680562
Epoch 6000: 0.0032954588532447815
Epoch 6500: 0.0029181884601712227
Epoch 7000: 0.002608819864690304
Epoch 7500: 0.0023518360685557127
Epoch 8000: 0.0021412218920886517
Epoch 8500: 0.0019676643423736095
Epoch 9000: 0.0018202407518401742
Epoch 9500: 0.0016941301291808486


In [16]:
px.scatter(x = X.flatten(), y = [model311(X).flatten().detach(), y.flatten()])

3.3.2 Решить задачу регрессии, соблюдая следующие условия:

1. Оформить нейронную сеть в виде объекта `nn.Sequential`
2. При создании сети использовать готовые блоки из `torch.nn`: слои, функции активации, функции потерь и т.д.
3. Для оптимизации использовать любой алгоритм оптимизации из `torch.optim` 

In [17]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())

In [18]:
model_seq = torch.nn.Sequential(
    torch.nn.Linear(1, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 1)
)
optimizer = torch.optim.SGD(model_seq.parameters(), 0.01)
loss = torch.nn.MSELoss()

for epoch in range(10000):

    y_pred = model_seq(X)
    data_loss = loss(y_pred, y)
    data_loss.backward()

    with torch.no_grad():
        optimizer.step()
        model_seq.zero_grad()
            
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: {data_loss.item()}")


Epoch 0: 0.6875715851783752
Epoch 500: 0.14095854759216309
Epoch 1000: 0.08962499350309372
Epoch 1500: 0.05271526798605919
Epoch 2000: 0.03065142221748829
Epoch 2500: 0.018612965941429138
Epoch 3000: 0.012311446480453014
Epoch 3500: 0.009008562192320824
Epoch 4000: 0.007197158876806498
Epoch 4500: 0.006100284866988659
Epoch 5000: 0.005345687735825777
Epoch 5500: 0.004748103674501181
Epoch 6000: 0.004236140288412571
Epoch 6500: 0.003801838494837284
Epoch 7000: 0.003431697143241763
Epoch 7500: 0.003109896555542946
Epoch 8000: 0.00282771117053926
Epoch 8500: 0.002591764321550727
Epoch 9000: 0.0023896140046417713
Epoch 9500: 0.002218574518337846


In [19]:
px.scatter(x = X.flatten(), y = [model_seq(X).flatten().detach(), y.flatten()])

## 3.4. Datasets and dataloaders

In [20]:
from torch.utils.data import Dataset, DataLoader

3.4.1 Создать датасет, поставляющий данные из задачи 3.1.2. 

Создать `DataLoader` на основе этого датасета и проверить работоспособность.

Воспользовавшись результатами 3.3.1 (или 3.3.2) обучите модель, пользуясь мини-пакетным градиентным спуском с размером пакета (`batch_size`) = 10

In [21]:

class SinDataset(Dataset):
    def __init__(self, inp: torch.Tensor, target: torch.Tensor):
        self.inp, self.tgt = inp, target

    def __len__(self):
        return self.inp.shape[0]

    def __getitem__(self, idx):
        return self.inp[idx], self.tgt[idx]


X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())
dataset = SinDataset(X, y)


model = Model331()
optimizer = torch.optim.SGD(model.parameters(), 0.01)
loss = torch.nn.MSELoss()

for i in range(1001):
    for x, y in DataLoader(dataset, batch_size=10):
        y_pred = model(x)
        data_loss = loss(y_pred.flatten(), y.flatten())
        data_loss.backward()

        with torch.no_grad():
            optimizer.step()
            model.zero_grad()

    if i % 500 == 0:
        print(f"Epoch {i}: {data_loss.item()}")


Epoch 0: 7.363915920257568
Epoch 500: 0.0004124066326767206
Epoch 1000: 0.00037041856558062136


3.4.2 Предсказание цен алмазов

3.4.2.1 Создайте датасет на основе файла diamonds.csv. 

1. Удалите все нечисловые столбцы
2. Целевой столбец (`y`) - `price`
3. Преобразуйте данные в тензоры корректных размеров

3.4.2.2 Разбейте датасет на обучающий и тестовый датасет при помощи `torch.utils.data.random_split`.

3.4.2.3 Обучите модель для предсказания цен при помощи мини-пакетного градиентного спуска (`batch_size = 256`). 

3.4.2.4 Выведите график функции потерь в зависимости от номера эпохи (значение потерь для эпохи рассчитывайте как среднее значение ошибок на каждом батче). Проверьте качество модели на тестовой выборке. 


In [22]:
import pandas as pd

# 3.4.2.1
class DiamondsDataset(Dataset):
    def __init__(self, data: pd.DataFrame, target: pd.Series):
        self.data = torch.tensor(data.values).float()
        self.target = torch.tensor(target.values).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

data = pd.read_csv("diamonds.csv", index_col=0)
dataset = DiamondsDataset(data.drop(columns=["cut", "color", "clarity", "y"]), data.y)


In [23]:
# 3.4.2.2
test_size = int(len(dataset) * 0.2)
train_data, test_data = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

In [24]:
# Данные очень большие из-за этого числа выходят за пределы float32 и возникает nan.
# Построим модель с батч-нормализацией, чтобы избежать этого.

In [25]:
model = torch.nn.Sequential(
    torch.nn.BatchNorm1d(6),
    torch.nn.Linear(6, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 1)
)

In [26]:
# 3.4.2.3
optimizer = torch.optim.Adam(model.parameters(), 0.001)
loss = torch.nn.MSELoss()
losses = []


for i in range(51):
    epoch_loss = 0
    for j, (x, y) in enumerate(DataLoader(train_data, batch_size=256), 1):
        y_pred = model(x)
        data_loss = loss(y_pred.flatten(), y.flatten())
        data_loss.backward()
        epoch_loss += data_loss.item()
        with torch.no_grad():
            optimizer.step()
            model.zero_grad()

    losses.append(epoch_loss / j)
    if i % 10 == 0:
        print(f"Epoch {i}: {epoch_loss / j:.4f}")


Epoch 0: 7.3214
Epoch 10: 0.0761
Epoch 20: 0.0630
Epoch 30: 0.0547
Epoch 40: 0.0487
Epoch 50: 0.0438


In [27]:
px.line(x=range(51), y=losses, labels={"x": "Epoch", "y": "Loss"})

In [28]:
# 3.4.2.4
model.eval()
test_pred = model(test_data[:][0])
test_loss = loss(test_pred.flatten(), test_data[:][1].flatten()).item()
print(f"Test loss: {test_loss:.4f}")

Test loss: 4.7598


3.4.3 Модифицируйте метод `__init__` датасета из 3.4.2 таким образом, чтобы он мог принимать параметр `transform: callable`. Реализуйте класс `DropColsTransform` для удаления нечисловых данных из массива. Реализуйте класс `ToTensorTransorm` для трансформации массива в тензор.

In [34]:
class DiamondsDataset(Dataset):
    def __init__(self, data, transform: callable = None):
        self.data = data
        self.X = data.drop(columns=["y"])
        self.transform = transform
        self.y = data.y

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = self.X.iloc[idx], self.y.iloc[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

In [51]:
class DropColsTransform:
    def __init__(self, drop: list):
        self.drop = drop

    def __call__(self, sample):
        _X, _y = sample
        # <удаление из X столбцов self.drop>
        _X: pd.Series
        _X = _X.drop(self.drop)
        return _X.values, _y

In [52]:
class ToTensorTransform:
    def __call__(self, sample):
        _X, _y = sample
        # <преобразование X и y в тензоры>
        return torch.tensor(list(_X)).float(), torch.tensor(_y).float()

In [53]:

from torchvision import transforms

drop = DropColsTransform(drop=["cut", "color", "clarity"])
to_tensor = ToTensorTransform()
dataset = DiamondsDataset(data, transforms.Compose([drop, to_tensor]))
train_data, test_data = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

In [ ]:
# 3.4.2.3
model = torch.nn.Sequential(
    torch.nn.BatchNorm1d(6),
    torch.nn.Linear(6, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 1)
)

optimizer = torch.optim.Adam(model.parameters(), 0.01)
loss = torch.nn.MSELoss()
losses = []


for i in range(11):
    epoch_loss = 0
    for j, (x, y) in enumerate(DataLoader(train_data, batch_size=4096), 1):
        y_pred = model(x)
        data_loss = loss(y_pred.flatten(), y.flatten())
        data_loss.backward()
        epoch_loss += data_loss.item()

        with torch.no_grad():
            optimizer.step()
            model.zero_grad()

    losses.append(epoch_loss / j)
    if i % 2 == 0:
        print(f"Epoch {i}: {epoch_loss / j:.4f}")

Epoch 0: 17.5993
Epoch 2: 1.8745
